In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import h5py
from nd2_to_caiman import np_arr_from_nd2
import labrotation.file_handling as fh
from matplotlib import pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
import matplotlib.colors as mcolors
from math import floor, ceil, sqrt, atan2, acos, pi, sin, cos
from datetime import datetime
import json
from labrotation import json_util
import scipy
from scipy import ndimage
from scipy.spatial import distance_matrix
from scipy.stats import circmean, circstd  # for statistical testing on directionality
import datadoc_util
from statsmodels.nonparametric.smoothers_lowess import lowess
import pandas as pd
import seaborn as sns
import multiprocess as mp  # multiprocessing does not work with IPython. Use fork instead.
import os
import random  # for surrogate algorithm
from collections.abc import Iterable
import math
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")
color_palette = sns.color_palette("deep")

In [ ]:
def get_datetime_for_fname():
    now = datetime.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
grid_shape = (8,8)

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "DATA_DOCU_FOLDER" in env_dict.keys():
    data_docu_folder = env_dict["DATA_DOCU_FOLDER"]
else:
    data_docu_folder = fh.open_dir("Open Data Documentation folder")
print(data_docu_folder)

In [ ]:
ddoc = datadoc_util.DataDocumentation(data_docu_folder)
ddoc.loadDataDoc()

In [ ]:
output_folder = env_dict["DOWNLOADS_FOLDER"]
print(f"Output files will be saved to {output_folder}")

# Open grid and traces for SD-spiking recording
T333 tmev d2 FOV007, FOV009. UUID: `06ebcf354f5c41519669f187e16de364`, `73a27053f4bf4ae1b4ad96064b6dabc0`

## Open grid data

In [ ]:
uuid_lis = ["06ebcf354f5c41519669f187e16de364", "73a27053f4bf4ae1b4ad96064b6dabc0"]

In [ ]:
ddoc.getSegmentsForUUID(uuid_lis[0])

In [ ]:
df_id_uuid = ddoc.getIdUuid()

In [ ]:
df_id_uuid = df_id_uuid[df_id_uuid["uuid"].isin(uuid_lis)]

In [ ]:
# grid files have nikon file name (without extension) as root. 
input_fname_roots = list(map(lambda rec_name: os.path.splitext(rec_name)[0], ddoc.getNikonFileNameForUuid(uuid_lis)))

In [ ]:
analysis_folder = fh.open_dir("Open directory with analysis (grid) data for all mice!")

In [ ]:
grid_files_list = []
for root, dirs, files in os.walk(analysis_folder):
    for fname in files:
        if "_grid.h5" in fname:
            # check if root file name is in the current _grid.h5 file
            if any([fname_root for fname_root in input_fname_roots if fname_root in fname]):
                grid_files_list.append(os.path.join(root,fname))

In [ ]:
uuid_dict = dict()
for grid_fpath in grid_files_list:
    # ..._grid.h5 -> ..._cnmf.hdf5
    cnmf_fpath = os.path.join(os.path.split(grid_fpath)[0], os.path.split(grid_fpath)[-1][:-7] + "cnmf.hdf5")
    with h5py.File(cnmf_fpath, 'r') as hf:
        uuid_dict[grid_fpath] = hf.attrs["uuid"]

In [ ]:
cols_set = set()
for fpath in grid_files_list:
    df = pd.read_hdf(fpath)
    for key in df.keys():
        cols_set.add(key)
cols_set.add("uuid")
cols_set.add("mouse_id")

In [ ]:
uuid_dict

In [ ]:
# defining empty dataframe does not work, as all data types will be object (except x, y, which will be proper integers)
all_onsets_df = pd.read_hdf(grid_files_list[0])
all_onsets_df["uuid"] = uuid_dict[grid_files_list[0]]
all_onsets_df["mouse_id"] = df_id_uuid[df_id_uuid["uuid"] == uuid_dict[grid_files_list[0]]]["mouse_id"].values[0]
assert all_onsets_df["uuid"].isna().sum() == 0
for fpath in grid_files_list[1:]:
    df = pd.read_hdf(fpath)
    df["uuid"] = uuid_dict[fpath]
    df["mouse_id"] = df_id_uuid[df_id_uuid["uuid"] == uuid_dict[fpath]]["mouse_id"].values[0]
    assert df["uuid"].isna().sum() == 0
    all_onsets_df = pd.concat([all_onsets_df, df])

## Open traces

In [ ]:
traces_dict = dict()
for grid_fpath in grid_files_list:
    # ..._grid.h5 -> ..._traces.hdf5
    traces_fpath = os.path.join(os.path.split(grid_fpath)[0], os.path.split(grid_fpath)[-1][:-7] + "traces.h5")
    with h5py.File(traces_fpath, 'r') as hf:
        uuid = hf.attrs["uuid"]
        traces_dict[uuid] = hf["traces"][:]

# Get neuron indices

x (horizontal): 0 is left, 512 is right

y (vertical): 0 is top, 512 is bottom

In [ ]:
neuron_ids_dict = {"06ebcf354f5c41519669f187e16de364": {"sd_only": [], "spikes": [] }, "73a27053f4bf4ae1b4ad96064b6dabc0": {"sd_only": [], "spikes": []},}

neuron_ids_dict["06ebcf354f5c41519669f187e16de364"]["sd_only"] = all_onsets_df[(all_onsets_df["uuid"] == "06ebcf354f5c41519669f187e16de364") & (all_onsets_df["x"] > 300.) & (all_onsets_df["y"] < 300. )].neuron_id.to_numpy()
neuron_ids_dict["06ebcf354f5c41519669f187e16de364"]["spikes"] = all_onsets_df[(all_onsets_df["uuid"] == "06ebcf354f5c41519669f187e16de364") & (all_onsets_df["x"] > 110.) & (all_onsets_df["y"] > 310. )].neuron_id.to_numpy()

neuron_ids_dict["73a27053f4bf4ae1b4ad96064b6dabc0"]["sd_only"] = all_onsets_df[(all_onsets_df["uuid"] == "73a27053f4bf4ae1b4ad96064b6dabc0") & (all_onsets_df["x"] > 280.) & (all_onsets_df["y"] < 210. )].neuron_id.to_numpy()
neuron_ids_dict["73a27053f4bf4ae1b4ad96064b6dabc0"]["spikes"] = all_onsets_df[(all_onsets_df["uuid"] == "73a27053f4bf4ae1b4ad96064b6dabc0") & (all_onsets_df["x"] > 140.) & (all_onsets_df["y"] > 210. )].neuron_id.to_numpy()

In [ ]:
begin_end_frames_dict = {"06ebcf354f5c41519669f187e16de364": []}

In [ ]:
mean_traces_dict = {}
for uuid in neuron_ids_dict:
    mean_traces_dict[uuid] = dict()
    for ival_type in neuron_ids_dict[uuid].keys():
        ival_ids = neuron_ids_dict[uuid][ival_type]
        mean_traces_dict[uuid][ival_type] = traces_dict[uuid][ival_ids].mean(axis=0)

In [ ]:
fig = plt.figure(figsize=(18,12))
plt.plot(mean_traces_dict["06ebcf354f5c41519669f187e16de364"]["sd_only"][5600:6000])
plt.plot(mean_traces_dict["06ebcf354f5c41519669f187e16de364"]["spikes"][5600:6000])
#plt.savefig("D:\\Downloads\\flashing_over_sd.jpg")
plt.show()

In [ ]:
all_ids = np.sort(np.concatenate([neuron_ids_dict["06ebcf354f5c41519669f187e16de364"]["sd_only"], neuron_ids_dict["06ebcf354f5c41519669f187e16de364"]["spikes"]]))
a = np.mean(traces_dict["06ebcf354f5c41519669f187e16de364"][all_ids], axis=0)[5600:6000]
fig = plt.figure(figsize=(18, 12))
plt.plot(a)
plt.savefig("D:\\Downloads\\flashing_over_sd_mean.jpg")
plt.show()

In [ ]:
all_ids = np.sort(np.concatenate([neuron_ids_dict["06ebcf354f5c41519669f187e16de364"]["sd_only"], neuron_ids_dict["06ebcf354f5c41519669f187e16de364"]["spikes"]]))
a = np.mean(traces_dict["06ebcf354f5c41519669f187e16de364"], axis=0)[5600:6000]
fig = plt.figure(figsize=(18, 12))
plt.plot(a)
plt.savefig("D:\\Downloads\\flashing_over_sd_all_in_FOV.jpg")
plt.show()